In [20]:
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
#from IEKF import IEKF

np.random.seed(1234567)

# Grid construction in [0,1] by [0,1] 
xgrid = np.linspace(0, 1, 21)
ygrid = np.linspace(0, 1, 21)

KK = 30 # Number of each sin/cos basis components


# Solution of the first order PDE at (x,y) 
def PDE_SOL(x,y,uu):
    sin_term_x = []
    cos_term_x = []
    
    sin_term_xy = []
    cos_term_xy = []
    
    for k in range(KK):
        sin_term_x.append(np.sin((k+1)*np.pi*x)*(1/((k+1)*np.pi)))
        cos_term_x.append(-np.cos((k+1)*np.pi*x)*(1/((k+1)*np.pi)))
        
        sin_term_xy.append(np.sin((k+1)*np.pi*(x+y))*(1/((k+1)*np.pi)))
        cos_term_xy.append(-np.cos((k+1)*np.pi*(x+y))*(1/((k+1)*np.pi)))
    
    init_cond = np.cos(x+y)
    
    trig_term_x = np.concatenate((np.array(sin_term_x), np.array(cos_term_x)))
    trig_term_xy = np.concatenate((np.array(sin_term_xy), np.array(cos_term_xy)))
    
    return init_cond*np.exp(np.dot(trig_term_x, uu)-np.dot(trig_term_xy, uu))

# Forward map
def h_u(uu):
    outcome_mat = np.zeros((len(xgrid), len(ygrid)))
    for i in range(len(xgrid)):
        x = xgrid[i]
        for j in range(len(ygrid)):
            y = ygrid[j]
            outcome_mat[i, j] = PDE_SOL(x, y, uu)
    outcome_mat[2,5] = 100
    return outcome_mat.flatten()

# True parameter
true_u = np.zeros(KK*2)
true_u[0] = 0.4
true_u[2] = 0.4
true_u[5] = -0.4
true_u[KK] = -0.2
true_u[KK+2] = -0.4
true_u[KK+5] = 0.2

true_u = true_u*3

# Data construction
noiseless_dat= h_u(true_u)
sigma = 0.1
y = noiseless_dat + np.random.normal(0, sigma, len(xgrid)**2)

In [23]:
noiseless_dat[10]

0.8775825618903728

In [8]:
noiseless_dat

array([ 1.        ,  0.93782051,  0.86472803,  0.79878015,  0.77290235,
        0.8074239 ,  0.89546338,  0.99253864,  1.03064415,  0.97874247,
        0.87758256,  0.79054114,  0.74929454,  0.74793168,  0.75908916,
        0.75777427,  0.74550891,  0.74969546,  0.79938589,  0.90156709,
        1.02121521,  0.99875026,  0.920909  ,  0.85067651,  0.82311745,
        0.85988184,  0.9536412 ,  1.0570234 ,  1.0976046 ,  1.0423309 ,
        0.93459868,  0.84190222,  0.79797585,  0.79652444,  0.80840682,
        0.8070065 ,  0.79394426,  0.79840282,  0.85132161,  0.96014147,
        1.08756307,  1.1517241 ,  0.99500417,  0.91912086,  0.88934443,
        0.92906684,  1.03036997,  1.14207016,  1.18591647,  1.12619552,
        1.0097953 ,  0.9096406 ,  0.86217997,  0.86061178,  0.87345021,
        0.87193722,  0.85782401,  0.86264129,  0.91981786,  1.03739324,
        1.17506702,  1.24439036,  1.17879546,  0.98877108,  0.95673821,
        0.99947075,  1.10845055,  1.22861529,  1.27578424,  1.21